In [1]:
import json
import os
import mp
import numpy as np
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import soundfile as sf
import re

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
def chunk(alignment, reject = -7, minimum_length = 1.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    temp[-1]['end'] = float(temp[-1]['end']) + 0.1
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            temp[-1]['end'] = float(temp[-1]['end']) + 0.1
            alls.append(temp)
    return alls

In [4]:
data = defaultdict(list)
with open('prepared-pseudolabel.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data[l['audio_filename']].append((no, l))
        
len(data)

3085595it [00:10, 297855.77it/s]


1961155

In [5]:
rows = list(data.values())
len(rows)

1961155

In [6]:
!rm -rf /home/husein/ssd3/prepared-pseudolabel-chunks
!mkdir /home/husein/ssd3/prepared-pseudolabel-chunks

In [7]:
def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        
        aligns, scores = [], []
        for i in d:
            f = f'prepared-pseudolabel_alignment/{i[0]}.alignment'
            try:
                with open(f) as fopen:
                    align = json.load(fopen)
                    score = np.sum([s['score'] for s in align])
                    aligns.append(align)
                    scores.append(score)
            except:
                aligns.append([])
                scores.append(-9999)

        argmax = np.argmax(scores)
        no = d[argmax][0]
        text = d[argmax][1]['new_text']
        lang = text.split('<|startoftranscript|><|')[1].split('|')[0]
        cleaned_text = re.sub(r"<\|.*?\|>", "", text).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        chunks = chunk(aligns[argmax])
        audio_filename = d[argmax][1]['audio_filename']
        if len(chunks):
            y, sr = sf.read(audio_filename)
            for k, c in enumerate(chunks):
                
                y_ = y[int(sr * c[0]['start']): int(sr * c[-1]['end'])]
                
                skip = False
                
                for c_ in c:
                    if (c_['end'] - c_['start']) > 2:
                        skip = True
                        break
                if skip:
                    continue
                
                for no_ in range(len(c)):
                    if no_ > 0 and (c[no_]['start'] - c[no_ - 1]['end']) > 1.2:
                        skip = True
                        break
                if skip:
                    continue
                    
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                    
                new_f = os.path.join('/home/husein/ssd3/prepared-pseudolabel-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y_, sr)
                    
                word = ''.join(ts)
                word = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{word}<|endoftext|>"
                
                segments, temp = [], [c[0]]
                last_t = c[0]['end']
                for c_ in c[1:]:
                    if (c_['start'] - last_t) > 0.25:
                        segments.append(temp)
                        temp = []

                    last_t = c_['end']
                    temp.append(c_)

                if len(temp):
                    segments.append(temp)
                
                ts = []
                for s in segments:
                    start = min(timestamps, key=lambda t: abs(t - (s[0]['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (s[-1]['end'] - min_t)))
                    w = ' '.join([c_['text'] for c_ in s])
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                
                ts = ''.join(ts)
                new_text = f"<|startoftranscript|><|{predict_lang}|><|transcribe|>{ts}<|endoftext|>"
                
                new_data.append({
                    'audio_filename': new_f,
                    'word_timestamp': word,
                    'segment_timestamp': new_text,
                })
                
    return new_data

In [8]:
r = loop((rows[:10], 0))
len(r)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 14.83it/s]


45

In [10]:
r[-10]

{'audio_filename': '/home/husein/ssd3/prepared-pseudolabel-chunks/12-4.mp3',
 'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Saya<|0.12|><|0.18|> tak<|0.26|><|0.30|> cerita<|0.54|><|0.60|> lagi<|0.78|><|0.86|> kita<|1.02|><|1.06|> ada<|1.12|><|1.22|> minimum<|1.56|><|1.64|> rest.<|1.98|><|2.66|> Tak<|2.74|><|2.78|> boleh<|2.92|><|2.96|> keluar<|3.14|><|3.18|> rumah.<|3.44|><|3.56|> Yang<|3.66|><|3.72|> tu<|3.86|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Saya tak cerita lagi kita ada minimum rest.<|1.98|><|2.66|> Tak boleh keluar rumah. Yang tu<|3.86|><|endoftext|>'}

In [11]:
import IPython.display as ipd
ipd.Audio(r[-10]['audio_filename'])

In [12]:
r = mp.multiprocessing(rows, loop, cores = 20)

100%|███████████████████████████████████| 98057/98057 [4:11:41<00:00,  6.49it/s]


In [14]:
len(r)

5944743

In [15]:
r[-10]

{'audio_filename': '/home/husein/ssd3/prepared-pseudolabel-chunks/3085589-4.mp3',
 'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Saya<|0.12|><|0.18|> tak<|0.26|><|0.30|> cerita<|0.54|><|0.60|> lagi<|0.78|><|0.86|> kita<|1.02|><|1.06|> ada<|1.12|><|1.22|> minimum<|1.56|><|1.64|> rest.<|1.98|><|2.66|> Tak<|2.74|><|2.78|> boleh<|2.92|><|2.96|> keluar<|3.14|><|3.18|> rumah.<|3.44|><|3.56|> Yang<|3.66|><|3.72|> tu<|3.86|><|endoftext|>',
 'segment_timestamp': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Saya tak cerita lagi kita ada minimum rest.<|1.98|><|2.66|> Tak boleh keluar rumah. Yang tu<|3.86|><|endoftext|>'}

In [16]:
import IPython.display as ipd
ipd.Audio(r[-10]['audio_filename'])

In [17]:
import copy
import re

results = []
for r_ in tqdm(r):
    r_ = copy.copy(r_)
    r_['audio_filename'] = r_['audio_filename'].replace('/home/husein/ssd3/', '').replace('/home/husein/ssd4/', '')
    clean_text = re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip().split()
    ratio = (len([w for w in clean_text if len(w) <= 1]) / len(clean_text))
    if ratio > 0.2:
        continue
    results.append(r_)

100%|████████████████████████████████████████████████████████████████████████████████████| 5944743/5944743 [00:18<00:00, 326204.11it/s]


In [18]:
from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [19]:
filtered = []
for r_ in tqdm(results):
    if skip_trigrams(re.sub(r"<\|.*?\|>", "", r_['segment_timestamp']).strip()):
        continue
    filtered.append(r_)
len(filtered)

100%|████████████████████████████████████████████████████████████████████████████████████| 5820101/5820101 [00:28<00:00, 204299.07it/s]


3305115

In [20]:
for r_ in filtered:
    if len(r_['word_timestamp']) > 2000:
        print(r_)
        break

{'audio_filename': 'prepared-pseudolabel-chunks/31733-0.mp3', 'word_timestamp': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Selemasa<|0.52|><|0.84|> yang<|0.94|><|1.00|> lebih<|1.16|><|1.20|> pantas<|1.60|><|1.78|> Latensi<|2.16|><|2.26|> dia<|2.32|><|2.38|> rendah<|2.66|><|2.86|> Jadi<|3.02|><|3.22|> selama<|3.46|><|3.54|> ni<|3.58|><|3.66|> Kalau<|3.82|><|3.88|> orang<|4.02|><|4.06|> yang<|4.14|><|4.18|> main<|4.30|><|4.40|> game<|4.56|><|4.66|> Dota<|4.96|><|5.12|> ke<|5.18|><|5.30|> apakah<|5.66|><|5.80|> Dia<|5.86|><|5.92|> akan<|6.12|><|6.42|> nak<|6.52|><|6.58|> dapatkan<|6.96|><|7.08|> capaian<|7.42|><|7.50|> internet<|7.76|><|7.98|> Yang<|8.16|><|8.40|> berlatensi<|9.04|><|9.14|> rendah<|9.38|><|9.54|> Bila<|9.66|><|9.98|> selemasa<|10.44|><|10.54|> tu<|10.60|><|10.68|> rendah<|10.94|><|11.08|> Dia<|11.16|><|11.22|> akan<|11.38|><|11.44|> dapat<|11.72|><|11.98|> Sampaikan<|12.52|><|12.72|> data<|12.98|><|13.10|> dia<|13.20|><|13.32|> Kepada<|13.60|><|13.78|> oran

In [21]:
len(filtered) / len(results)

0.567879320307328

In [22]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('pseudolabel-whisper-word-timestamp.parquet')

In [23]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-whisper-word-timestamp.parquet",
    path_in_repo="data/malaysian_context_v2-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

pseudolabel-whisper-word-timestamp.parquet:   0%|          | 0.00/808M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/ecfbd0b02c4b15674a3cb168e47c0e719952a01b', commit_message='Upload data/malaysian_context_v2-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='ecfbd0b02c4b15674a3cb168e47c0e719952a01b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)